In [0]:
!pip install transformers
import tensorflow as tf
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Dense, Dropout
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint
from transformers import DistilBertTokenizer, RobertaTokenizer

In [0]:
train = pd.read_csv("/content/drive/My Drive/Kaggle_disaster/train.csv")
test = pd.read_csv("/content/drive/My Drive/Kaggle_disaster/test.csv")

In [0]:
roberta = 'distilbert-base-uncased'
tokenizer = DistilBertTokenizer.from_pretrained(roberta, do_lower_case = True, add_special_tokens = True, max_length = 128, pad_to_max_length = True)

In [0]:
def tokenize(sentences, tokenizer):
  input_ids, input_masks, input_segments = [], [], []
  for sentence in sentences:
    inputs = tokenizer.encode_plus(sentence, add_special_tokens = True, max_length = 128, pad_to_max_length = True, return_attention_mask = True, return_token_type_ids = True)
    input_ids.append(inputs['input_ids'])
    input_masks.append(inputs['attention_mask'])
    input_segments.append(inputs['token_type_ids'])
  return np.asarray(input_ids, dtype = "int32"), np.asarray(input_masks, dtype = "int32"), np.asarray(input_segments, dtype = "int32")

In [0]:
input_ids, input_masks, input_segments = tokenize(train.text.values, tokenizer)

In [0]:
from transformers import TFDistilBertForSequenceClassification, DistilBertConfig, TFDistilBertModel

distil_bert = 'distilbert-base-uncased'

config = DistilBertConfig(dropout=0.2, attention_dropout=0.2)
config.output_hidden_states = False
transformer_model = TFDistilBertModel.from_pretrained(distil_bert, config = config)

input_ids_in = tf.keras.layers.Input(shape=(128,), name='input_token', dtype=tf.int32)
input_masks_in = tf.keras.layers.Input(shape=(128,), name='masked_token', dtype=tf.int32) 
embedding_layer = transformer_model(input_ids_in, attention_mask=input_masks_in)[0]
X = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50, return_sequences=True, dropout=0.1, recurrent_dropout=0.1))(embedding_layer)
X = tf.keras.layers.GlobalMaxPool1D()(X)
X = tf.keras.layers.Dense(50, activation='relu')(X)
X = tf.keras.layers.Dropout(0.2)(X)
X = tf.keras.layers.Dense(1, activation='sigmoid')(X)
model = tf.keras.Model(inputs=[input_ids_in, input_masks_in], outputs = X)
model.compile(Adam(lr = 1e-5), loss = 'binary_crossentropy', metrics = ['accuracy'])
for layer in model.layers[:3]:
  layer.trainable = False


In [0]:
model.summary()

Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_token (InputLayer)        [(None, 128)]        0                                            
__________________________________________________________________________________________________
masked_token (InputLayer)       [(None, 128)]        0                                            
__________________________________________________________________________________________________
tf_distil_bert_model_2 (TFDisti ((None, 128, 768),)  66362880    input_token[0][0]                
__________________________________________________________________________________________________
tf_op_layer_strided_slice_1 (Te [(None, 768)]        0           tf_distil_bert_model_2[0][0]     
____________________________________________________________________________________________

In [0]:
bert_input = [
    input_ids,
    input_masks
]
train_history = model.fit(
    bert_input,
    train.target.values,
    validation_split = 0.2,
    batch_size = 16,
    epochs = 10
)

Epoch 1/10
164/164 [==============================] - 51s 311ms/step - loss: 0.7983 - accuracy: 0.5103 - val_loss: 0.7266 - val_accuracy: 0.5436
Epoch 2/10
164/164 [==============================] - 50s 304ms/step - loss: 0.7334 - accuracy: 0.5621 - val_loss: 0.7159 - val_accuracy: 0.5605
Epoch 3/10
164/164 [==============================] - 50s 304ms/step - loss: 0.6942 - accuracy: 0.5954 - val_loss: 0.7237 - val_accuracy: 0.5544
Epoch 4/10
164/164 [==============================] - 50s 304ms/step - loss: 0.6946 - accuracy: 0.5927 - val_loss: 0.7192 - val_accuracy: 0.5804
Epoch 5/10
164/164 [==============================] - 50s 304ms/step - loss: 0.6557 - accuracy: 0.6487 - val_loss: 0.7430 - val_accuracy: 0.5590
Epoch 6/10
164/164 [==============================] - 50s 303ms/step - loss: 0.6251 - accuracy: 0.6609 - val_loss: 0.7511 - val_accuracy: 0.5605
Epoch 7/10
164/164 [==============================] - 50s 304ms/step - loss: 0.6002 - accuracy: 0.6874 - val_loss: 0.7756 - val_ac

In [0]:
input_ids, input_masks, input_segments = tokenize(test.text.values, tokenizer)
bert_input = [
    input_ids,
    input_masks
]
submission = pd.read_csv("/content/sample_submission.csv")
test_pred = model.predict(bert_input)
submission['target'] = test_pred.round().astype(int)
submission.to_csv('submission.csv', index=False)

In [0]:
model.save_weights('/content/drive/My Drive/disaster_model/model_hugging_face1.h5')